In [1]:
from datetime import datetime

from utils import get_context, get_response

# **Structured Output Generation using XML Formatting**

This notebook demonstrates how to generate **structured output** in **XML format** for downstream processing and automation workflows. It ensures that generated responses are **machine-readable, structured, and consistent**, improving **interoperability** with external systems.

## **Overview**

- Implements **structured response generation** using **XML format**.
- Enhances **data organization and automation** by ensuring consistent output formatting.
- The notebook workflow includes:
  - Extracting **key information** from user input.
  - Structuring the extracted data into a **well-defined XML format**.
  - Ensuring **validity** and **consistency** of XML output for seamless integration.
- Provides an **example-driven approach** to XML generation, demonstrating step-by-step output formatting.

The structured approach enables better **data exchange**, **automation**, and **interoperability** in service-oriented architectures.


In [2]:
system_instruction_prompt = """
    You are a helpful AI assistant designed to assist users in scheduling meetings.

    # Your Purpose
    1. You will be given a request from a user to schedule a meeting.
    2. Your job is to extract the parts of the request that are relevant to scheduling the meeting.
    3. Use Local Date: as a reference only when a specific date is not provided in the user's request.

    # Valid Parts
    - Time Duration: The duration of the meeting in minutes.
    - Date: The date of the meeting.
    - Participants: List of participants of the meeting.
    - Meeting Topic: The main topic or subject of the meeting.
    - Additional Details: Any additional context or details mentioned.

    # Example Input
    Schedule 30 minutes on Friday with Masha and Matt to talk about Product.
    Schedule half an hour on Friday with Masha and Matt to talk about Product.

    # Example Output
    <time_duration>30</time_duration>
    <date>2024-08-09</date>
    <participants> [Masha, Matt] </participants>
    <meeting_topic>Product</meeting_topic>
    <additional_details>talk about Product</additional_details>

    # Notes on the date:
    - The date should be in the format "YYYY-MM-DD".
    - If a specific date is provided in the user's request (e.g., "09/13/2024"), use that date and convert it to the YYYY-MM-DD format.
    - If the date is not provided, return today's date.
    - If the date just says "Friday," calculate this week's Friday from the current date.
    - If the date says "next Friday," calculate next week's Friday from the current date.
    - Ensure that the calculated date is in the future. If "Friday" refers to a day in the past based on the current date, assume it refers to the next occurrence of that day.
    - Always return a future date for relative terms like "Friday" or "next Friday."

    # Notes on time duration:
    - The time duration should be in minutes.
    - The user might say "30 minutes" or "half an hour," which should be converted to minutes.
    - The user might say "1 hour," which should be converted to 60 minutes.
    - If a specific time range is provided (e.g., "10:00am-10:30am"), calculate the duration in minutes.
    - If the time duration is not provided, assume it is 30 minutes.

    # Output Format - XML
    You will respond using XML tags for each extracted piece of information. You don't need to provide explanation or any other information, just return the extracted parts within the appropriate XML tags.

    # Format of the Output
    <time_duration>extracted_time_duration</time_duration>
    <date>extracted_date</date>
    <participants>[participant1, participant2]</participants>
    <meeting_topic>extracted_meeting_topic</meeting_topic>
    <additional_details>extracted_additional_details</additional_details>
"""

## Test Scenario: Basic Meeting Extraction

In [3]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'

In [4]:
async def test_extract_meeting_invite_parts_basic_scenario():
    """
    Scenario: The user provides straightforward info (a single participant,
    direct date, direct time duration and a topic) to schedule a meeting.
    Expected Outcome: correct information in every component.
    """
    print("Starting test: test_extract_meeting_invite_parts_basic_scenario")

    user_message = "Schedule a meeting with Matt on 09/13/2024 for 10:00am-10:30am to discuss Products."
    today = datetime.now().strftime("%Y-%m-%d")

    message = f"{user_message} Today's date: {today}"
    context = get_context(system_instruction_prompt, message, user_input_label="Schedule meeting request")

    expected_output = {
        "time_duration": 30,
        "date": "2024-09-13",
        "participants": ["Matt"],
        "meeting_topic": "Products",
        "additional_details": "discuss Products",
    }

    extracted_parts = await get_response(context, ["time_duration", "date", "participants", "meeting_topic", "additional_details"])
    extracted_parts['time_duration'] = int(extracted_parts['time_duration'])
    
    print(f"{color.BOLD} Extracted parts: {color.END} {extracted_parts}")

    assert (
        expected_output == extracted_parts
    ), f"Extracted parts do not match the expected output. Expected: {expected_output}, Got: {extracted_parts}"

    print("Test completed successfully! 🙌")

# Run the test
await test_extract_meeting_invite_parts_basic_scenario()

Starting test: test_extract_meeting_invite_parts_basic_scenario


INFO:inference:Inflection AI API request took 3582.82 ms


 Extracted parts:  {'time_duration': 30, 'date': '2024-09-13', 'participants': ['Matt'], 'meeting_topic': 'Products', 'additional_details': 'discuss Products'}
Test completed successfully! 🙌
